In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.cleaning import get_types_col


In [2]:
df_base = pd.read_csv('../data/training_inputs.csv', sep=';', decimal='.')
y = pd.read_csv('../data/training_outputs.csv', sep=';', decimal='.')
df_base = pd.merge(df_base, y, on='ID', how='inner')  

In [3]:
df = df_base.copy()

## 1: Traitement des données

--- 

Première étape pour un projet de data science: Nettoyer les données

In [4]:
df_base["Q66"].unique()

array(['true', 'false', nan, 'true '], dtype=object)

In [5]:
df, types = get_types_col(df)

Premier nettoiement des données et séparation des différents types de variables (qualitatives, quantitatives etc...)

#### Booléennes

In [11]:
df[types[0]].head(5)

,C5,C6,C7,Q7,Q16,Q52,Q53,Q54,Q55,Q56,...,Q70,Q71,Q72,Q73,Q74,Q75,C14,C9,C4,C8
0,False,False,False,False,False,NaN,False,NaN,NaN,NaN,...,True,False,False,NaN,NaN,NaN,True,True,True,True
1,False,False,False,NaN,False,False,True,True,False,True,...,True,False,False,True,False,False,True,True,False,False
2,False,False,False,NaN,False,NaN,False,False,False,True,...,True,False,False,NaN,NaN,NaN,False,True,False,False
3,False,False,False,NaN,True,NaN,False,False,False,False,...,True,False,False,NaN,NaN,NaN,True,True,True,False
4,False,False,False,False,False,NaN,False,NaN,NaN,NaN,...,True,False,False,NaN,NaN,NaN,False,True,True,True


#### Qualitatives

In [12]:
df[types[1]].head(5)

,C3,Q1,Q11,C13,Q39,Q3,Q28,Q12,Q34,C12,Q29,C2,Q32,Q10,Q36,Q2
0,C,HA,D7,RC,NaN,UC,ZA,EA,ED,OB,IA,RA,LC,PC,AC,ZA
1,C,HC,DS,RC,NaN,UC,ZF,E8,ED,OB,ID,RA,LC,PC,AC,ZA
2,B,HC,D3,R7,NaN,UA,ZF,EA,EF,OC,IA,RC,LA,PD,AD,ZA
3,C,HC,DB,RC,NaN,UC,ZA,EA,ED,OA,IA,RA,LD,PA,AC,ZA
4,B,HA,DF,RC,NaN,UA,ZA,EA,ED,OB,IA,RA,LD,PC,AC,ZA


#### Quantitatives

In [13]:
df[types[2]].head(5)

,ID,C10,C11,C15,C16,C17,C18,C19,S1,S2,...,Q42,Q43,Q44,Q45,Q46,Q47,Q48,Q49,Q50,Q51
0,0,8,2286,2,214,212,16,7,1.0,0,...,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0
1,1,166,2220,3,511,414,15,19,3.0,2,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
2,2,17,2265,4,407,405,17,8,NaN,0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3,3,16,2250,3,712,410,16,12,4.0,1,...,0.0,0.0,1.0,1.0,1.0,1.0,3.0,2.0,2.0,3.0
4,4,23,2305,1,206,204,17,4,NaN,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


#### Temporelles

In [14]:
df[types[-1]].head(5)

,S3,S4,S5,S6,S7
0,2013-11-15,2014-02-22,2013-09-08,2013-09-07,2013-09-07
1,2013-11-15,2014-08-07,2013-04-18,2013-09-14,2014-03-21
2,2013-11-15,2013-11-05,2013-11-05,NaT,NaT
3,2013-11-15,2014-05-30,2013-07-03,2014-01-30,2014-05-18
4,2013-11-15,2013-09-22,2013-09-22,NaT,NaT


### Présentation du problème

On cherche à prédire la variable "TARGET", qui contient 20% de 1 (pour 80% de 0)

In [15]:
df["TARGET"].mean()

0.19359515485975518

Il est évident que dans cette configuration, certaines métriques comme l'accuracy ne sont pas très pertinentes

In [17]:
from sklearn.metrics import f1_score, roc_auc_score, classification_report

In [18]:
print(classification_report(df["TARGET"].to_numpy(), [0]*df.shape[0]))

              precision    recall  f1-score   support

       False       0.81      1.00      0.89     68971
        True       0.00      0.00      0.00     16558

    accuracy                           0.81     85529
   macro avg       0.40      0.50      0.45     85529
weighted avg       0.65      0.81      0.72     85529



/home/onyxia/work/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/onyxia/work/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/onyxia/work/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


D'autres métriques telles que le F1 score ou le ROC AUC sont plus pertinentes

In [19]:
roc_auc_score(df["TARGET"].to_numpy(), [0]*df.shape[0])

0.5

In [20]:
float(roc_auc_score(df["TARGET"].to_numpy(), np.random.randint(0, 2, size=df.shape[0])))

0.4992292984370987

In [21]:
f1_score(df["TARGET"].to_numpy(), np.random.randint(0, 2, size=df.shape[0]))

0.2790854408518812

#### Gérer les NAN

Pour les variables catégorielles, on se permet de remplir simplement par "Unknown" les variables avec des NaN

In [ ]:
df.fillna({col: "Unknown" for col in types[1]}, inplace=True)
df[types[1]].head(5)

,C3,Q1,Q11,C13,Q39,Q3,Q28,Q12,Q34,C12,Q29,C2,Q32,Q10,Q36,Q2
0,C,HA,D7,RC,Unknown,UC,ZA,EA,ED,OB,IA,RA,LC,PC,AC,ZA
1,C,HC,DS,RC,Unknown,UC,ZF,E8,ED,OB,ID,RA,LC,PC,AC,ZA
2,B,HC,D3,R7,Unknown,UA,ZF,EA,EF,OC,IA,RC,LA,PD,AD,ZA
3,C,HC,DB,RC,Unknown,UC,ZA,EA,ED,OA,IA,RA,LD,PA,AC,ZA
4,B,HA,DF,RC,Unknown,UA,ZA,EA,ED,OB,IA,RA,LD,PC,AC,ZA


In [55]:
list_types = ["Bool", "Qualitative", "Quantitative", "Temporelle"]

for cat, list_col in zip(list_types, types):
    if cat == "Qualitative":
        continue

    missing_rate = df[list_col].isna().mean()
    print(cat, "\n", f"Nombre de variables : {len(list_col)}\n", "% de colonnes qui contiennent:\n")
    print(f" Au moins un Nan {len(missing_rate[missing_rate > 0.].index)/len(list_col)}")
    print(f" Au moins 10% de NaN {len(missing_rate[missing_rate > 0.1].index)/len(list_col)}")
    print(f" Au moins 50% de NaN: {len(missing_rate[missing_rate > 0.5].index)/len(list_col)}")
    print(f" Au moins 80% de NaN {len(missing_rate[missing_rate > 0.8].index)/len(list_col)}")
    print('\n', "-"*50, "\n")

Bool 
 Nombre de variables : 33
 % de colonnes qui contiennent:

 Au moins un Nan 0.7878787878787878
 Au moins 10% de NaN 0.2727272727272727
 Au moins 50% de NaN: 0.06060606060606061
 Au moins 80% de NaN 0.0

 -------------------------------------------------- 

Quantitative 
 Nombre de variables : 52
 % de colonnes qui contiennent:

 Au moins un Nan 0.7115384615384616
 Au moins 10% de NaN 0.19230769230769232
 Au moins 50% de NaN: 0.1346153846153846
 Au moins 80% de NaN 0.09615384615384616

 -------------------------------------------------- 

Temporelle 
 Nombre de variables : 5
 % de colonnes qui contiennent:

 Au moins un Nan 0.4
 Au moins 10% de NaN 0.4
 Au moins 50% de NaN: 0.0
 Au moins 80% de NaN 0.0

 -------------------------------------------------- 

